In [34]:
import duckdb

In [52]:
# set the query
query = f""" SELECT * FROM players; """

In [53]:
# get data from db
conn = duckdb.connect('../transfermarkt.db')
try:
    df = conn.sql(query).df()
    # results = conn.execute(query).fetchall()
except Exception as e:
    print(f"Error: {e}")
finally:
    conn.close()

In [57]:
df.query("player_id == '192279'")['player_img_url'].values[0]

'https://img.a.transfermarkt.technology/portrait/header/192279-1661855851.jpg?lm=1'

In [46]:
df.query("player_id == '258919'").sort_values(by='season', ascending=True)

,id,player_id,season,fee,from_club,to_club,transfer_date,from_club_image_url,to_club_image_url
17,18,258919,13/14,-,Man Utd Youth,Man Utd U18,01/07/2013,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/hea...
15,16,258919,15/16,loan transfer,Man Utd U21,Stockport,12/01/2016,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/hea...
14,15,258919,15/16,End of loan,Stockport,Man Utd U21,20/02/2016,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/hea...
13,14,258919,15/16,loan transfer,Man Utd U21,Stockport,24/03/2016,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/hea...
12,13,258919,15/16,End of loan,Stockport,Man Utd U21,31/05/2016,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/hea...
16,17,258919,15/16,-,Man Utd U18,Man Utd U21,01/07/2015,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/hea...
11,12,258919,16/17,loan transfer,Man Utd U23,Grimsby Town,31/08/2016,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/hea...
10,11,258919,16/17,End of loan,Grimsby Town,Man Utd U23,03/02/2017,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/hea...
9,10,258919,17/18,loan transfer,Man Utd U23,Shrewsbury,10/07/2017,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/hea...
8,9,258919,17/18,End of loan,Shrewsbury,Man Utd U23,31/05/2018,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/hea...
